In [22]:
import os
import logging
import pickle
import osmnx as ox
import numpy as np
import pandas as pd
import geopandas as gpd
import h3
import shapely
import requests
from tqdm.notebook import tqdm
from pyproj import Transformer
from shapely.ops import linemerge
from shapely import Polygon
from pyogrio.errors import DataSourceError
import plotly.express as px

ox.settings.cache_folder = './data'

## functions

In [4]:
def fetch_google_places(categories:list[str], lon:float, lat:float, radius:float, apikey:str):
    
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": apikey,
        "X-Goog-FieldMask": "places.id,places.displayName,places.location,places.businessStatus,places.formattedAddress,places.primaryTypeDisplayName,places.primaryType",
    }
    url = "https://places.googleapis.com/v1/places:searchNearby"

    data = {
            "includedTypes": categories,
            "maxResultCount": 20,
            "locationRestriction": {
                "circle": {
                    "center": {"latitude": lat, "longitude": lon},
                    "radius": radius,
                }
            },
        }
    
    r = requests.post(url, headers=headers, json=data)
    if r.status_code != 200:
        logging.warning(f"error at request: {r.request}")
        return

    data = r.json().get('places', [])
    if not data:
        return data

    if len(data) >= 20:
        logging.warning(f'query at lon:{lon}, lat:{lat} returned 20+ results')
    
    for r in data:
        r['name'] = r['displayName']['text']
        r['geometry'] = shapely.Point(r['location']['longitude'], r['location']['latitude'])

    return data



In [5]:
def h3cell_to_shapely(cell):
    coords = h3.cell_to_boundary(cell)
    flipped = tuple(coord[::-1] for coord in coords)
    return Polygon(flipped)

## inputs

In [6]:
apikey = os.getenv("GOOGLE_API_KEY")
radius = 200
step = 100
buffer = 30
categories = ['cafe', 'bakery']
map_style = "open-street-map"

# Geospatial Analysis

## download network (roads)

In [7]:
bbox = [23.7576, 37.8524, 23.7729, 37.899]
g = ox.graph_from_bbox(bbox) # we can modify filters
nodes, links = ox.graph_to_gdfs(g)

## create the linestring geometry

In [8]:
road = links[links['name'] == 'Δημητρίου Γούναρη']
crs = road.estimate_utm_crs()
geom_proj = road.to_crs(crs).union_all()
geoms_proj = gpd.GeoDataFrame(geometry=[linemerge(geom_proj)]).explode()
geoms_proj['length'] = geoms_proj.length
geom_proj = geoms_proj.sort_values(['length'], ascending=False).iloc[0].geometry
road_buffered = road.to_crs(crs).buffer(buffer).to_crs(epsg=4326).union_all()

In [9]:
fp = './data/cafes.gpkg'
length = geom_proj.length

try:
    gdf = gpd.read_file(fp)
except DataSourceError:
    places = []
    transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)
    for distance in tqdm(np.arange(0, length, step)):
        p = shapely.line_interpolate_point(geom_proj, distance) # type:ignore
        p = transformer.transform(p.x, p.y)
        p = shapely.Point(p[0], p[1])
        r = fetch_google_places(categories, p.x, p.y, radius, apikey)
        if r is not None:
            places.extend(r)

    with open('./data/places.pkl', 'wb') as f:
        pickle.dump(places, f)
    
    gdf = gpd.GeoDataFrame(places, crs='epsg:4326').drop_duplicates(subset=['id'])
    gdf['inbuffer'] = gdf.intersects(road_buffered) # keep cafes up to 30 meters from the road (inside the buffer)
    gdf.to_file(fp)    

In [12]:
gdf['inbuffer'] = gdf.intersects(road_buffered) # keep cafes up to 30 meters from the road (inside the buffer)
cafes = gdf[gdf['inbuffer']]
cafes.head(2)

,id,formattedAddress,location,businessStatus,displayName,primaryTypeDisplayName,primaryType,name,geometry,inbuffer
1,ChIJgcEWYU6_oRQR-cd2bf6di68,"Attikis 1, Glifada 165 62, Greece","{'latitude': 37.894874, 'longitude': 23.761325...",OPERATIONAL,"{'text': 'coffee berry - Άνω Γλυφάδα', 'langua...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}",coffee_shop,coffee berry - Άνω Γλυφάδα,POINT (23.76133 37.89487),True
2,ChIJgSMfxKu_oRQRDDje9yAhZfM,"Dim. Gounari 24, Elliniko 167 77, Greece","{'latitude': 37.893558399999996, 'longitude': ...",OPERATIONAL,"{'text': 'ROSEWOOD COFFEE HOUSE', 'languageCod...","{'text': 'Coffee Shop', 'languageCode': 'en-US'}",coffee_shop,ROSEWOOD COFFEE HOUSE,POINT (23.76089 37.89356),True


## stats / visuals

In [13]:
fig = px.scatter_map(gdf, lat=gdf.geometry.y, lon=gdf.geometry.x, color='inbuffer', text='name', hover_name="name", zoom=12, map_style=map_style)
fig.show()

In [14]:
fig = px.density_map(cafes, lat=cafes.geometry.y, lon=cafes.geometry.x, radius=12, zoom=12, map_style=map_style, width=600)
fig.show()

In [15]:
f'One cafe every: {length / cafes.shape[0]:.0f}m'

'One cafe every: 125m'

In [16]:
# Distance between cafes (distance matrix)

In [32]:
gdf = cafes.to_crs(crs) # project the coordinates

dists = []
for i, g in gdf.iterrows():
    geom = g.geometry
    d = gdf.distance(geom).tolist()
    dists.append(d)

dists = pd.DataFrame(dists, index=gdf.index, columns=gdf.index).round(2)
dists

,1,2,3,8,9,10,14,15,16,17,...,39,40,44,45,46,47,48,50,51,52
1,0.00,150.93,32.59,216.53,312.53,396.85,822.68,747.69,1100.24,1089.75,...,2350.19,2361.13,2474.07,2731.11,2756.83,2892.25,2936.57,3191.30,3145.88,3234.03
2,150.93,0.00,151.69,67.89,164.76,250.26,678.12,602.27,958.73,948.32,...,2209.48,2221.41,2333.40,2590.97,2616.98,2753.75,2799.14,3055.33,3009.77,3098.25
3,32.59,151.69,0.00,219.36,316.27,401.58,829.11,753.50,1108.62,1098.18,...,2359.21,2370.68,2483.12,2740.46,2766.34,2902.47,2947.32,3202.74,3157.26,3245.56
8,216.53,67.89,219.36,0.00,96.92,182.37,610.23,534.38,891.00,880.61,...,2141.76,2153.78,2265.68,2523.29,2549.33,2686.25,2731.79,2988.17,2942.60,3031.13
9,312.53,164.76,316.27,96.92,0.00,85.59,513.51,437.55,794.92,784.56,...,2045.59,2057.86,2169.51,2427.22,2453.34,2590.60,2636.45,2893.25,2847.64,2936.26
10,396.85,250.26,401.58,182.37,85.59,0.00,427.92,352.01,709.49,699.14,...,1960.10,1972.47,2084.01,2341.76,2367.91,2505.33,2551.33,2808.37,2762.73,2851.40
14,822.68,678.12,829.11,610.23,513.51,427.92,0.00,76.82,284.71,274.70,...,1533.16,1546.31,1657.04,1915.05,1941.40,2079.91,2126.98,2385.45,2339.68,2428.67
15,747.69,602.27,753.50,534.38,437.55,352.01,76.82,0.00,361.45,351.40,...,1609.83,1623.08,1733.71,1991.75,2018.13,2156.71,2203.80,2462.24,2416.48,2505.46
16,1100.24,958.73,1108.62,891.00,794.92,709.49,284.71,361.45,0.00,10.57,...,1250.76,1262.99,1374.68,1632.31,1658.42,1796.08,1842.68,2100.84,2055.09,2144.04
17,1089.75,948.32,1098.18,880.61,784.56,699.14,274.70,351.40,10.57,0.00,...,1261.16,1273.32,1385.08,1642.69,1668.78,1806.36,1852.89,2110.97,2065.22,2154.16


In [38]:
nearests = gdf.sjoin_nearest(gdf, distance_col='sdist', exclusive=True)[['id_left', 'id_right', 'sdist']]
nearests
nearests['sdist'].mean()

np.float64(50.49189035272862)

### spatial grouping (h3)

In [ ]:
box = shapely.box(*bbox)
coords = [list(box.exterior.coords)]
cells = h3.geo_to_cells(box, res=9)
cells = [{'h3id': cell, 'geometry': h3cell_to_shapely(cell)} for cell in cells]
cells = gpd.GeoDataFrame(cells, crs='epsg:4326')
cells.head(2)
px.choropleth_map(cells, geojson=cells.geometry, locations=cells.index,
                  center={'lat': box.centroid.y, 'lon': box.centroid.x}, zoom=11.5,
                  map_style=map_style)

In [ ]:
counts = gpd.sjoin(cells, cafes, predicate='contains').groupby(['h3id', 'geometry'], as_index=False)['index_right'].count()
counts = counts.rename(columns={'index_right':'count'})
counts = gpd.GeoDataFrame(counts) # groupby casts to pd.DataFrame
px.choropleth_map(counts, geojson=counts.geometry, locations=counts.index,
                  color='count', color_continuous_scale=px.colors.sequential.Reds,
                  center={'lat': box.centroid.y, 'lon': box.centroid.x}, zoom=11.5,
                  title='Concentration of cafes (H3 cells)', map_style=map_style)